In [1]:
%pip install maritalk
%pip install pyttsx3
%pip install SpeechRecognition
%pip install pyaudio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from maritalk import MariTalk
import datetime
import pyttsx3
import speech_recognition as sr

with open('key_txt.txt', 'r') as file:
    maritalk_key = file.read().strip()

maritalk_client = MariTalk(key=maritalk_key, model="sabia-2-small")

def listen_and_transcribe():
    recognizer = sr.Recognizer()
    
    with sr.Microphone() as source:
        print("Ajustando para barulho de ambiente...")
        recognizer.adjust_for_ambient_noise(source)
        print("Escutando...")
        audio = recognizer.listen(source)
    
    try:
        print("Transcrevendo...")
        text = recognizer.recognize_google(audio, language='pt-BR')
        print(f"Transcrição: {text}")
        
        return text

    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand the audio")
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")


def text_to_speech(text, lang='pt'):
    engine = pyttsx3.init()
    
    engine.setProperty('rate', 200)  # Speed percent
    engine.setProperty('volume', 1)  # Volume 0-1

    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)

    engine.say(text)
    
    engine.runAndWait()

def update_sensor_data(sensor_data):
    context_message = {
        "role": "system",
        "content": f"Responda o usuário com um bom dia/tarde/noite baseado no horario atual, além disso sempre que o usuário" + 
        f" quiser saber a temperatura ou a hora atual cheque as informações a seguir. Contexto de sensores: {sensor_data}" + 
        f", não faça formatação no texto de resposta."
    }
    return context_message

def receive_message(history, message):
    history.append({"role": "user", "content": message})
    return history

def generate_response(history):
    print_history(history)
    response = maritalk_client.generate(
        history,
        do_sample=True,
        max_tokens=1000,
        temperature=0.7,
        top_p=0.95
    )["answer"]
    
    history.append({"role": "assistant", "content": response})
    return response, history

def print_history(history):
    for entry in history:
        if entry["role"] == "system":
            print(f"System: {entry['content']}")
        elif entry["role"] == "user":
            print(f"You: {entry['content']}")
        elif entry["role"] == "assistant":
            print(f"Bot: {entry['content']}")

def chatbot_loop():
    time = datetime.datetime.now().strftime("%H:%M")
    sensor_data = {'temperatura_quarto': 23, 'temperatura_sala': 24, 'horario_agora': time}

    history = [update_sensor_data(sensor_data)]

    while 1:
        print('')
        user_message = listen_and_transcribe()
        if 'ASSISTENTE' in user_message.upper():
            history = receive_message(history, user_message)
            response, history = generate_response(history)
            print(f"Bot: {response}")
            text_to_speech(response)

            time = datetime.datetime.now().strftime("%H:%M")
            sensor_data = {'temperatura_quarto': 24, 'temperatura_sala': 26, 'horario_agora': time}
            context_message = update_sensor_data(sensor_data)

            history = [context_message] + history[1:]
        elif 'PARAR CONVERSA' in user_message.upper():
            break
        else:
            continue

In [2]:
chatbot_loop()


Ajustando para barulho de ambiente...
Escutando...
Transcrevendo...
Transcrição: assistente Quantos graus está agora na sala enquanto eu estou indo lá me dê uma receita de bolo de chocolate
System: Responda o usuário com um bom dia/tarde/noite baseado no horario atual, além disso sempre que o usuário quiser saber a temperatura ou a hora atual cheque as informações a seguir. Contexto de sensores: {'temperatura_quarto': 23, 'temperatura_sala': 24, 'horario_agora': '12:50'}, não faça formatação no texto de resposta.
You: assistente Quantos graus está agora na sala enquanto eu estou indo lá me dê uma receita de bolo de chocolate
Bot: Bom dia! A temperatura na sala agora está em 24 graus. Quanto à receita de bolo de chocolate, aqui vai uma bem simples e deliciosa:

Ingredientes:
- 3 ovos
- 2 xícaras de farinha de trigo
- 2 xícaras de açúcar
- 1 xícara de cacau em pó
- 1 xícara de leite
- 1/2 xícara de óleo
- 1 colher de sopa de fermento em pó
- 1 pitada de sal
- Chocolate granulado para de